In [16]:
# eda.ipynb 
# behöver inte lämnas in
# from functools import lru_cache as cache

# Ignore cell during run all
#%%script false --no-raise-error

import pandas as pd 



# @cache
def readin(file, drop=[]):
    return pd.read_csv(f"data/{file}.csv").drop(columns=drop)
#   return pd.read_csv(f"data/tags.csv").drop(columns=drop)
    
ratings = readin("ratings", drop=["timestamp"])
movies  = readin("movies")
tags    = readin("tags", drop=["timestamp"]) 
print(
    f"ratings : {ratings.shape}"
    f"movies  : {movies.shape}"
    f"tags    : {tags.shape}"
)

ratings : (33832162, 3)movies  : (86537, 3)tags    : (2328315, 3)


In [17]:
ratings.shape
ratings.describe()

,userId,movieId,rating
count,3.383216e+07,3.383216e+07,3.383216e+07
mean,1.654380e+05,2.831348e+04,3.542540e+00
std,9.534122e+04,4.992865e+04,1.063959e+00
min,1.000000e+00,1.000000e+00,5.000000e-01
25%,8.295300e+04,1.219000e+03,3.000000e+00
50%,1.661290e+05,3.263000e+03,4.000000e+00
75%,2.474500e+05,4.049100e+04,4.000000e+00
max,3.309750e+05,2.889830e+05,5.000000e+00


In [18]:
ratings.groupby("movieId").mean()

,userId,rating
movieId,,
1,165293.494604,3.893508
2,164501.015393,3.278179
3,164859.653856,3.171271
4,166063.999339,2.868395
5,164789.207012,3.076957
...,...,...
288967,47791.000000,3.500000
288971,98408.000000,0.500000
288975,154483.000000,4.000000


In [19]:
# Collect the least reviewd movies aka all movies with less then 10 ratings
print(f"Number of unique movies: {len(ratings["movieId"].value_counts())}")
n = 100
print(f"Number of unique movies with reviews greater then {n}: {len( [obj for obj in ratings["movieId"].value_counts()if obj > n] )}")
li = [f"\n{obj}: {val}" for obj, val in ratings["movieId"].value_counts().items() if val > n]
#print(*li)

keep = [obj for obj, val in ratings["movieId"].value_counts().items() if val > n]
ratings_r = movies[ movies["movieId"].isin(keep) ]



Number of unique movies: 83239
Number of unique movies with reviews greater then 100: 12206


In [20]:
# This list contains all the movies to keep
movieId_master_list = keep

In [21]:
# Collect the movies with the lowest score aka all movies with a score less then 4

In [22]:
# Depricated

# Collect the least reviewd movies aka all movies with less then 10 ratings
movieId_few_ratings = ratings.groupby("movieId").count()
movieId_few_ratings = movieId_few_ratings[ movieId_few_ratings["userId"]<10]
print(f"Movies with ratings < 10   : {movieId_few_ratings.shape[0]}")

# Collect the movies with the lowest score aka all movies with a score less then 4
movieId_low_score = ratings.groupby("movieId").mean()
movieId_low_score = movieId_low_score[ movieId_low_score["rating"]<4]
print(f"Movies with a score < 4    : {movieId_low_score.shape[0]}")

# Collect the most controversial movies aka the movies with the most 5, 4.5 and 1.5, 1, 0.5
# ratings_sample[ (ratings_sample["rating"]<2) | (ratings_sample["rating"]>4) ]

movieId_most_high_scores = ratings[ (ratings["rating"]>4) ]
movieId_most_high_scores = movieId_most_high_scores.groupby("movieId").count()
movieId_most_high_scores = movieId_most_high_scores[ movieId_most_high_scores["rating"]>1] # Shuld be like 10
movieId_most_high_scores

movieId_most_low_scores  = ratings[ (ratings["rating"]<2) ]
movieId_most_low_scores = movieId_most_low_scores.groupby("movieId").count()
movieId_most_low_scores = movieId_most_low_scores[ movieId_most_low_scores["rating"]>2]
movieId_most_low_scores["rating"] = movieId_most_low_scores["rating"].map( lambda x: x*1 )

high_std = movieId_most_high_scores 
print(high_std.shape)
high_std = high_std[high_std.index.isin(movieId_most_low_scores.index)]
print(high_std.shape)

low_std = movieId_most_low_scores 
print(low_std.shape)
low_std = low_std[low_std.index.isin(movieId_most_high_scores.index)]
print(low_std.shape)

movie_std = high_std["rating"]+low_std["rating"]
movie_std
# ratings_sample
# print(ratings_sample.shape)
# ratings_sample.drop(movieId_low_score.index, inplace=True)
# print( f"Original Rewiew set size          : {ratings_sample.shape[0]}" )
# new_ratings_sample = ratings_sample.copy(deep=True)
# new_ratings_sample = ratings_sample[~ratings_sample["movieId"].isin(movieId_low_score.index)]
# new_ratings_sample = new_ratings_sample[~new_ratings_sample["movieId"].isin(movieId_few_ratings.index)]
# print( f"New Rewiew set size               : {new_ratings_sample.shape[0]}" )

Movies with ratings < 10   : 51218
Movies with a score < 4    : 76043
(28374, 2)
(19279, 2)
(23875, 2)
(19279, 2)


movieId
1         27464
2          5275
3          2896
4           666
5          2737
          ...  
287609        8
287633       31
287731        6
288167       18
288311       27
Name: rating, Length: 19279, dtype: int64

In [23]:
# EDA Movies


# Expands geners by braking the genre column in to individual rows

expand = list()
for index, row in movies_genres.iterrows(): 
    mid = row["movieId"]
    genres = row["genres"]
    expand.extend( (mid, gen.lower()) for gen in genres.split("|")  )
    #print(row["movieId"],row["genres"])

movies_genres_exp = pd.DataFrame({"movieId":[tu[0] for tu in expand], "genre":[tu[1] for tu in expand]})
movies_genres_exp.head()


NameError: name 'movies_genres' is not defined

In [ ]:
print(movies_genres.shape)
print(movies_genres_exp.shape)

(86537, 2)
(152288, 2)


In [24]:
# EDA Tages
tags = pd.read_csv("data/tags.csv", usecols=lambda x: x not in ["timestamp","userId"])
tags.shape



(2328315, 2)

In [25]:
# Set all tags to lower case
tags["tag"] = tags["tag"].astype(str)
tags["tag"] = tags["tag"].map(lambda x: x.lower())
tags.head()

# Reduce the tags to the most common ones
# Probably shoud combine some tags by hand that are duplicates
print(tags.shape)
gp_tag = tags.groupby("tag").count()
gp_tag = gp_tag[ gp_tag["movieId"] > 2000 ]
gp_tag.shape
gp_tag

print(tags.shape)
tags_new = tags[ tags["tag"].isin(gp_tag.index) ]
print(tags_new.shape)


(2328315, 2)
(2328315, 2)
(526054, 2)


In [ ]:
#tags[ tags["movieId"] == ]

In [26]:
# Combine genre and tag
# Maybe you should could the tags for each movie and use the frequency to 
# adjust the weights, but for now I'm simply going with unique tags 
# so combine genre and tag and then remove dublicates for each movie
tags_new["tag"].unique()

# Why is the tag_genre shape smaller then the two that was merged?
mgt = movies_genres_exp.rename(columns={"genre":"tag"})
tag_genre = pd.merge(left=tags_new, right=mgt, how="outer")
#type(movies_genres.rename(columns={"genre":"tag"}))
print(mgt.shape[0]+tags_new.shape[0])
print(tag_genre.shape[0])
tag_genre_test = tag_genre.loc[0:1000].copy()

tag_genre_test.drop_duplicates( inplace=True)
tag_genre_test

NameError: name 'movies_genres_exp' is not defined

In [ ]:
mgt.head()

,movieId,tag
0,1,adventure
1,1,animation
2,1,children
3,1,comedy
4,1,fantasy


In [27]:


# 1) Counts the number of tags a movie have recieved
# this shoud be te reduced tags list
tags_di = { mid:{} for mid in tags["movieId"]}
for index, row in tags.head(10000).iterrows():
    tag = row["tag"]
    mid = row["movieId"]
    #print(row["movieId"], row["tag"])
    if tag in tags_di[mid].keys():
        tags_di[mid][tag]+=1 
    else: 
        tags_di[mid][tag]=1

# 2) Sorts the tags 
for key, di in tags_di.items():
    li = [ (key, val) for key, val in di.items() ]
    li.sort( key=lambda x: x[1], reverse=True )
    tags_di[key]=li
#print(tags_di.head(10))


#3) Adds the genre
#   Sould have removed the genres from the tags at an earlier state
for row in mgt.itertuples(index=False):
    if row.movieId in tags_di.keys():
        # (!) This if statetment is to catch movies without tags but 
        # it looks like all movies does have tags because no ** are 
        # added, but if I remove it, it breaks.
        # The * ** is simply to keep track, they should not be in the
        # final product
        if tags_di[row.movieId]:
            top = tags_di[row.movieId][0][1]
            tags_di[row.movieId].insert(0,(row.tag+"*",top))
        else:
            tags_di[row.movieId].append((row.tag+"**",1))

# 4) Limit the number of tags to max 7 
for key in tags_di.keys():
    tags_di[key] = tags_di[key][:13]
#print(tags_di.head(10))
tags_di
#mgt


NameError: name 'mgt' is not defined

In [ ]:
# Tags tf-idf

# 1) Remvoe all movieId from tag that is not in the movieId master list
tags_r = tags.copy()
tags_r[ tags_r["movieId"].isin(movieId_master_list)]

# 2) Reducera tags 
def reduction(df, column, n):
    print(f"Number of unique {column}s: {len(df[column].value_counts())}")
    print(f"Number of unique {column}s greater then {n}: {len( [counted_objects for counted_objects in df[column].value_counts()if counted_objects > n] )}")
    #li = [f"\n{tag}: {val}" for object, val in df[column].value_counts().items() if val > n]
    #print(*li)

    keep = [key for key, val in df[column].value_counts().items() if val > n]
    df = df[ df[column].isin(keep) ]
    return df
tags_r = reduction(tags_r, "tag", 100)
#print(len(tags_r["tag"].value_counts()))


# 3) Split movies in to titles and genres
# this way the exploded genres don't need to include titles
# and the program can save on memory
movies_r = movies.copy()
movies_r = movies_r[ movies_r["movieId"].isin(movieId_master_list)]

titles_r = movies_r.copy()
titles_r.drop(columns=["genres"], inplace=True)

genres_r = movies_r.copy()
genres_r.drop(columns=["title"], inplace=True)

# 4) Explode the genre column in Movies
def explode(df, id_column, explode_column, split_marker):
    expand = list()
    # Transforms the DataFrame in to a list with the exploded columns
    for index, row in df.iterrows(): 
        expand.extend( (row[id_column], gen.lower()) for gen in row[explode_column].split(split_marker)  )

    # Transforms the list back in to a DataFrame
    df_new = pd.DataFrame({id_column:[tu[0] for tu in expand], explode_column:[tu[1] for tu in expand]})
    return df_new

genres_r = explode(genres_r, "movieId", "genres", "|")


# 4) Weights 
# There are four types of weights to chose from 
# A) No weights: Each tag and Genere have the same weight 

# A.1) Counts the number of tags a movie have recieved
def subgroup_counter(df, group, subgroup):
    di = { group_key:{} for group_key in df[group]}
    for index, row in df.iterrows():
        if row[subgroup] in di[ row[group] ].keys():
            di[ row[group] ][ row[subgroup] ]+=1 
        else: 
            di[ row[group] ][ row[subgroup] ]=1
    return di

# (!) Remove .iloc[:] when deploying 
subgroup_counted = subgroup_counter(tags_r.iloc[:100000], "movieId", "tag")

# A.2) Adds the genres to the tags
#      How much impact the genres should have can be adjusted with impact
def add_subgroup():
    pass

# A.3) Sort the tags, set the limit if you want to limit the number of tags per movie 
def subgroup_sort(group_subgroup, limit=-1):
    di = { key:sorted( [ (so1_key, so1_val) for so1_key, so1_val in so1.items()][:limit], key=lambda x: x[1], reverse=True) for key, so1 in group_subgroup.items() } 
    di = { key:{ tu[0]:tu[1] for tu in so1 } for key, so1 in di.items() }
    return di
subgroup_counted_sorted = subgroup_sort(subgroup_counted, limit=5)


# A.3) 


# B) Term Frequency : Calculate the weight of the tags per movie
# Loop A: loops through the movies
# Scale : divides 1 by the sum of the total number of tags 
# Loop B: loops through the tags and multiplies there frequency with the scale 


# def term_frequency(group_subgroup):
#     for key, subgroups in group_subgroup.items():
#         li = list()
#         scale = 1/sum( tuple[1] for tuple in subgroups )
#         for tuple in subgroups:
#            li.append( (tuple[0], tuple[1]*scale) )
#         group_subgroup[key] = li
#     return group_subgroup

# tf_tags = term_frequency(subgroup_counted_sorted) 
# tf_tags


# C) Term Frequency - Inverse Document Frequency : Calclualte the tag weights based on all movies
# def inverse_document_frequency(group_subgroup):
#     frequency = [ group.keys() for group in group_subgroup]
#     documents = len(group_subgroup)
#     for key, subgroup in group_subgroup.items():
#         subgroup.keys()


# import math
# (1/7)*math.log(4/3)

# D) Genre Weights : Genre is not inculded in B and C because it would be dwarfed, 
#                    D allows you to adjust how much weight you wantthe Gnres to have


Number of unique tags: 143812
Number of unique tags greater then 100: 3242


{260: {'sci-fi': 100,
  'star wars': 24,
  'harrison ford': 21,
  'good vs evil': 15,
  'space epic': 9},
 1221: {'mafia': 7,
  'organized crime': 6,
  'al pacino': 5,
  'complex characters': 4,
  'classic': 3},
 58559: {'heath ledger': 19,
  'batman': 16,
  'dark': 15,
  'atmospheric': 14,
  'comic book': 9},
 57183: {'family': 1},
 296: {'quentin tarantino': 49,
  'dark comedy': 35,
  'cult film': 21,
  'crime': 15,
  'comedy': 7},
 47: {'twist ending': 21,
  'kevin spacey': 13,
  'morgan freeman': 13,
  'powerful ending': 12,
  'crime': 12},
 165: {'bruce willis': 6,
  'action': 4,
  'samuel l. jackson': 4,
  'sequel': 2,
  '1990s': 1},
 293: {'jean reno': 13,
  'gary oldman': 11,
  'natalie portman': 11,
  'great acting': 10,
  'loneliness': 5},
 480: {'dinosaurs': 12,
  'steven spielberg': 11,
  'action': 6,
  'scenic': 2,
  'classic': 1},
 527: {'holocaust': 20,
  'john williams': 2,
  'moving': 2,
  'acting': 1,
  'hope': 1},
 541: {'sci-fi': 22,
  'cyberpunk': 18,
  'dystopia':

In [ ]:
blue={ "A":{ "python":31, "giraffe":33, "sparrow":74 }, "B":{"python":63, "sparrow":23, "dog":4}, "C":{"cat":20, "python":24, "giraffe":13,  }, }

di = { key:sorted( [ (so1_key, so1_val) for so1_key, so1_val in so1.items()][:2], key=lambda x: x[1], reverse=True) for key, so1 in blue.items() } 
di = { key:{ tu[0]:tu[1] for tu in so1 } for key, so1 in di.items() }

di


{'A': {'giraffe': 33, 'python': 31},
 'B': {'python': 63, 'sparrow': 23},
 'C': {'python': 24, 'cat': 20}}

In [ ]:
genres_short = genres_r.iloc[:19].copy()

red={ 
    1:{"python":31, "giraffe":33, "sparrow":74}, 
    2:{"python":63, "sparrow":23, "dog":4}, 
    3:{"cat":20, "python":24, "giraffe":13}, 
    4:{"python":31, "giraffe":33, "sparrow":74}, 
    5:{"python":63, "sparrow":23, "dog":4}, 
    6:{"cat":20, "python":24, "giraffe":13}, 
    7:{"cat":20, "python":24, "giraffe":13}, 
    }

def add_genres_to_tags(di, df, impact=0.5):
    { key:sum(so1.values()) for key, so1 in di.items() }
    n_of_genres = df.groupby("movieId")["genres"].count()

    for idx, movie, genre in df.itertuples():
        di[movie][genre]=40
    return di 
#red = add_genres_to_tags(red, genres_short)

#for _ in genres_short.itertuples():
#    print(_[1])
#red
type(  genres_short.groupby("movieId")["genres"].count())
genres_short.groupby("movieId")["genres"].count().loc[1]

5

In [ ]:
#red={ "A":{ "hedgehog" "giraffe":33, "sparrow":74 }, "B":{"python":63, "sparrow":23, "dog":4}, "C":{"cat":20, "python":24, "giraffe":13,  }, }
#df_red = pd.DataFrame( data=red )
#def add_subgroup():

In [ ]:
import math
blue={ "A":{"dog":32, "cat":20, "python":24}, "B":{"python":24 }, "C":{"cat":20, "python":24 }, }


def idf_(docs): 
    li = list()
    for  document in docs.values():
        li += list(document.keys()) 
    return { val:math.log(len(docs)/li.count(val)) for val in set(li)}

def tf_idf_(docs):
    idf = idf_(docs)
    for doc_key, document in docs.items():
        total = sum(document.values())
        for term_key, term_frequency in document.items():
            docs[doc_key][term_key] = term_frequency * idf[term_key]
    return idf

tf_idf = tf_idf_(blue)
tf_idf

{'cat': 0.4054651081081644, 'dog': 1.0986122886681098, 'python': 0.0}

In [ ]:
red=['red','blue','red']
occurence = { val:red.count(val) for val in set(red)}
occurence

{'blue': 1, 'red': 2}

In [ ]:
print(movies.shape)
print(movies_r.shape)

(86537, 3)
(12206, 3)


In [ ]:
print(movies.memory_usage())
print(movies_r.memory_usage())
print(genres_r.memory_usage())
print(titles_r.memory_usage())

Index         132
movieId    692296
title      692296
genres     692296
dtype: int64
Index      97648
movieId    97648
title      97648
genres     97648
dtype: int64
Index         132
movieId    217360
genres     217360
dtype: int64
Index      97648
movieId    97648
title      97648
dtype: int64


In [ ]:
li = list(movies.columns)
li.remove("movieId")
li.remove("genres")
li

['title']

In [ ]:
# Reduce the number of tags
print(f"Number of unique tags: {len(tags["tag"].value_counts())}")
n = 100
print(f"Number of unique tags greater then {n}: {len( [tag for tag in tags["tag"].value_counts()if tag > n] )}")
li = [f"\n{tag}: {val}" for tag, val in tags["tag"].value_counts().items() if val > n]
print(*li)

# The actuall reduction part from n, 
# 1) makes a list of the tags that occures more then n number of times
keep = [tag for tag, val in tags["tag"].value_counts().items() if val > n]
print(tags.shape)
# 2) makes a new copy of tags that only keeps the tags in the keep list
tags_r = tags[ tags["tag"].isin(keep) ]
print(tags_r.shape)



Number of unique tags: 143812
Number of unique tags greater then 100: 3242

sci-fi: 14892 
atmospheric: 12776 
action: 11747 
comedy: 10966 
funny: 9851 
surreal: 9443 
visually appealing: 9086 
twist ending: 8428 
dark comedy: 7777 
thought-provoking: 7731 
based on a book: 7718 
romance: 7123 
dystopia: 7056 
cinematography: 6645 
fantasy: 6469 
social commentary: 6425 
violence: 6377 
stylized: 6201 
classic: 6182 
quirky: 6181 
psychology: 6136 
dark: 6127 
murder: 6032 
adventure: 5985 
time travel: 5784 
drama: 5520 
coming of age: 5369 
suspense: 5301 
thriller: 5244 
psychological: 5175 
friendship: 5145 
space: 5137 
revenge: 5099 
black comedy: 4968 
great soundtrack: 4905 
superhero: 4837 
nudity (topless): 4791 
cult film: 4728 
animation: 4717 
mystery: 4659 
horror: 4655 
based on a true story: 4645 
mindfuck: 4444 
disturbing: 4402 
great acting: 4362 
satire: 4359 
aliens: 4328 
true story: 4245 
predictable: 4220 
music: 4065 
crime: 4060 
drugs: 4015 
bd-r: 3969 
musi

In [ ]:
# Sandbox
df = pd.DataFrame({
    "Red":[200, 200, 400, 400, 500],
    "Blue":[200, 300, 400, 500, 600],
})

#idx = df[df["Red"]<500].index
#df.drop(df["Red"].isin([200,500]),inplace=True)
#df[~df["Red"].isin([200,500])]